## Just checking out this file to understand

In [1]:
import geopandas as gpd
import pandas as pd

from pathlib import Path
from typing import Literal

from calitp_data_analysis import utils
from shared_utils import publish_utils
from update_vars import GTFS_DATA_DICT 
  
PUBLIC_GCS = GTFS_DATA_DICT.gcs_paths.PUBLIC_GCS

In [4]:
digest_df_keys = [
        "route_schedule_vp", 
        "operator_profile_portfolio_view",  
        "operator_sched_rt",
        "scheduled_service_hours",
    ]  

In [2]:
PUBLIC_GCS

'gs://calitp-publish-data-analysis/'

In [7]:
table_section = GTFS_DATA_DICT["digest_tables"].dir

In [8]:
table_section

'gs://calitp-analytics-data/data-analyses/rt_vs_schedule/'

In [9]:
[GTFS_DATA_DICT[table_section][f] for f in digest_df_keys]

ConfigKeyError: Missing key gs://calitp-analytics-data/data-analyses/rt_vs_schedule/
    full_key: gs://calitp-analytics-data/data-analyses/rt_vs_schedule/
    object_type=dict